In [1]:
import numpy as np
import pandas as pd
import re
import json
import math
import sys
import os


In [26]:
root = '../src/baselines/runs/'
models = ['bert_oc_gs', 'bert_soc_gs']
datasets = ['davidson', 'founta', 'golbeck', 'harassment', 'hate']

In [40]:
# gridsearch aggregate
result = None
for model in models:
    for dataset in datasets:
        for subdir in os.listdir(f'{root}/{model}'):
            if os.path.exists(f'{root}/{model}/{subdir}/eval_results_0_dev_{dataset}.txt'):
                f = pd.read_csv(
                    filepath_or_buffer=f'{root}{model}/{subdir}/eval_results_0_dev_{dataset}.txt',
                    delimiter='=',
                    # names=['metric','score'],
                    header=None,
                )
                args = pd.read_json(f'{root}{model}/{subdir}/args.json', typ='series')
                args = args.to_frame().reset_index().rename(columns={"index": 0, 0: 1})
                f = pd.concat([f, args], ignore_index=True)
                f = f.append([['model', model[:-3]]], ignore_index=True)
                f = f.append([['dataset', dataset]], ignore_index=True)
                f.set_index(0, inplace=True)
                f = f.transpose()
                if result is None:
                    result = f
                else:
                    result = pd.concat([result, f], ignore_index=True, sort=False)
            else: pass


result.to_csv(f'{root}/gs_results/gridsearch_expl_results_raw.csv', index=False)

In [41]:
result


,acc,auc_roc,disparate_impact_favorable_06,disparate_impact_favorable_08,disparate_impact_unfavorable_06,disparate_impact_unfavorable_08,eval_loss,eval_loss_reg,f1,fnr_priv_06,...,local_rank,seed,gradient_accumulation_steps,fp16,loss_scale,server_ip,server_port,continue_from_checkpoint,model,dataset
0,0.93866,0.972684,0.236528,0.426022,1.26339,1.16326,0.168889,4.01722e-05,0.961771,0.0704762,...,-1,42,1,False,0,,,0,bert_oc,davidson
1,0.94431,0.97097,0.121867,0.0864928,1.30454,1.25488,0.158455,0.000468094,0.965396,0.0692063,...,-1,42,1,False,0,,,0,bert_oc,davidson
2,0.957627,0.981972,0.130753,0.0926209,1.2745,1.23213,0.13663,0.000138618,0.9739,0.0495238,...,-1,42,1,False,0,,,0,bert_oc,davidson
3,0.964487,0.987828,0.0964078,0.103284,1.25042,1.20043,0.100869,0.00132101,0.9784,0.0285714,...,-1,42,1,False,0,,,0,bert_oc,davidson
4,0.961259,0.986236,0.131026,0.0928088,1.27366,1.23149,0.107736,0.00142147,0.976143,0.0463492,...,-1,42,1,False,0,,,0,bert_oc,davidson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,0.939866,0.891043,1.02129,1.02923,0.292957,0,0.166691,0.00137342,0.389844,0.71875,...,-1,42,1,False,0,,,0,bert_soc,hate
81,0.940189,0.890779,1.02529,1.0313,0.21853,0,0.167264,0.00150361,0.404819,0.701923,...,-1,42,1,False,0,,,0,bert_soc,hate
82,0.94148,0.893815,1.01529,1.01515,0.509598,0.503864,0.164127,0.00110855,0.413905,0.699519,...,-1,42,1,False,0,,,0,bert_soc,hate
83,0.941561,0.895321,1.02177,1.03329,0.362816,0,0.164603,0.00127491,0.429022,0.679087,...,-1,42,1,False,0,,,0,bert_soc,hate


In [42]:
'bert_soc_gs'[:-3]

'bert_soc'

In [43]:
for model in models:
    for dataset in datasets:
        temp = result[(result['dataset']==dataset) & (result['model'] == model[:-3])]
        temp = temp.sort_values(by=['f1 '], ascending=False)
        temp.to_csv(f'{root}/gs_results/{model[:-3]}_{dataset}.csv', index=False)

In [44]:
best_results_gs = None
for model in models:
    for dataset in datasets:
        if best_results_gs is None:
            best_results_gs = pd.read_csv(f'{root}/gs_results/{model[:-3]}_{dataset}.csv').head(1)
        else:
           best_results_gs = pd.concat([best_results_gs, pd.read_csv(f'{root}/gs_results/{model[:-3]}_{dataset}.csv').head(1)])
best_results_gs.to_csv(f'{root}/gs_results/best_hp_results_expl.csv', index=False)

In [45]:
best_results_gs

,acc,auc_roc,disparate_impact_favorable_06,disparate_impact_favorable_08,disparate_impact_unfavorable_06,disparate_impact_unfavorable_08,eval_loss,eval_loss_reg,f1,fnr_priv_06,...,local_rank,seed,gradient_accumulation_steps,fp16,loss_scale,server_ip,server_port,continue_from_checkpoint,model,dataset
0,0.967312,0.988467,0.135910,0.103051,1.237363,1.201032,0.096138,0.001522,0.980113,0.026032,...,-1,42,1,False,0,NaN,NaN,0,bert_oc,davidson
0,0.939206,0.972975,0.185922,0.137582,3.260271,3.294739,0.177459,0.002693,0.889416,0.118084,...,-1,42,1,False,0,NaN,NaN,0,bert_oc,founta
0,0.791582,0.725867,0.934984,1.068872,1.948413,0.000000,0.480020,0.000165,0.318408,0.798301,...,-1,42,1,False,0,NaN,NaN,0,bert_oc,golbeck
0,0.903094,0.955676,0.083677,0.059509,2.810277,2.658547,0.257533,0.001100,0.872017,0.175424,...,-1,42,1,False,0,NaN,NaN,0,bert_oc,harassment
0,0.940189,0.889739,1.022384,1.024954,0.135665,0.000000,0.167939,0.001446,0.367208,0.743990,...,-1,42,1,False,0,NaN,NaN,0,bert_oc,hate
0,0.970137,0.989351,0.125620,0.102820,1.241612,1.201639,0.104425,0.000818,0.981827,0.024762,...,-1,42,1,False,0,NaN,NaN,0,bert_soc,davidson
0,0.938880,0.972734,0.176721,0.138058,3.257554,3.264810,0.176269,0.002505,0.889327,0.114368,...,-1,42,1,False,0,NaN,NaN,0,bert_soc,founta
0,0.784990,0.725321,0.583135,1.168346,3.507143,0.000000,0.515957,0.000545,0.442105,0.649682,...,-1,42,1,False,0,NaN,NaN,0,bert_soc,golbeck
0,0.908895,0.959597,0.086479,0.019939,2.775356,2.703913,0.245078,0.000653,0.880219,0.162342,...,-1,42,1,False,0,NaN,NaN,0,bert_soc,harassment
0,0.941561,0.895321,1.021770,1.033291,0.362816,0.000000,0.164603,0.001275,0.429022,0.679087,...,-1,42,1,False,0,NaN,NaN,0,bert_soc,hate


In [66]:
f = pd.read_csv(
                    filepath_or_buffer=f'{root}log_reg_results/glove_davidson_0/eval_results_dev_glove',
                    delimiter='=',
                    header=None,
                )
f[1] = f[1].astype(float)
print(f.dtypes)
args = pd.read_json(f'{root}log_reg_results/glove_davidson_0/args.json', typ='series', dtype=str)
args = args.to_frame().reset_index().rename(columns={"index": 0, 0: 1})
f = pd.concat([f, args], ignore_index=True)
f.set_index(0, inplace=True)
f = f.transpose()

f[['acc ', 'auc_roc ', 'disparate_impact_favorable_06 ',
       'disparate_impact_favorable_08 ', 'disparate_impact_unfavorable_06 ',
       'disparate_impact_unfavorable_08 ', 'f1 ', 'fnr_priv_06 ',
       'fnr_priv_08 ', 'fnr_total_06 ', 'fnr_total_08 ', 'fnr_unpriv_06 ',
       'fnr_unpriv_08 ', 'fpr_priv_06 ', 'fpr_priv_08 ', 'fpr_total_06 ',
       'fpr_total_08 ', 'fpr_unpriv_06 ', 'fpr_unpriv_08 ', 'loss ',
       'precision ', 'priv_n_06 ', 'priv_n_08 ', 'priv_ratio_favorable_06 ',
       'priv_ratio_favorable_08 ', 'priv_ratio_unfavorable_06 ',
       'priv_ratio_unfavorable_08 ', 'priv_total_06 ', 'priv_total_08 ',
       'recall ', 'unpriv_n_06 ', 'unpriv_n_08 ', 'unpriv_ratio_favorable_06 ',
       'unpriv_ratio_favorable_08 ', 'unpriv_ratio_unfavorable_06 ',
       'unpriv_ratio_unfavorable_08 ', 'unpriv_total_06 ', 'unpriv_total_08 ']] = f[['acc ', 'auc_roc ', 'disparate_impact_favorable_06 ',
       'disparate_impact_favorable_08 ', 'disparate_impact_unfavorable_06 ',
       'disparate_impact_unfavorable_08 ', 'f1 ', 'fnr_priv_06 ',
       'fnr_priv_08 ', 'fnr_total_06 ', 'fnr_total_08 ', 'fnr_unpriv_06 ',
       'fnr_unpriv_08 ', 'fpr_priv_06 ', 'fpr_priv_08 ', 'fpr_total_06 ',
       'fpr_total_08 ', 'fpr_unpriv_06 ', 'fpr_unpriv_08 ', 'loss ',
       'precision ', 'priv_n_06 ', 'priv_n_08 ', 'priv_ratio_favorable_06 ',
       'priv_ratio_favorable_08 ', 'priv_ratio_unfavorable_06 ',
       'priv_ratio_unfavorable_08 ', 'priv_total_06 ', 'priv_total_08 ',
       'recall ', 'unpriv_n_06 ', 'unpriv_n_08 ', 'unpriv_ratio_favorable_06 ',
       'unpriv_ratio_favorable_08 ', 'unpriv_ratio_unfavorable_06 ',
       'unpriv_ratio_unfavorable_08 ', 'unpriv_total_06 ', 'unpriv_total_08 ',]].astype(float)

print(f.dtypes)

0     object
1    float64
dtype: object
0
acc                                 float64
auc_roc                             float64
disparate_impact_favorable_06       float64
disparate_impact_favorable_08       float64
disparate_impact_unfavorable_06     float64
disparate_impact_unfavorable_08     float64
f1                                  float64
fnr_priv_06                         float64
fnr_priv_08                         float64
fnr_total_06                        float64
fnr_total_08                        float64
fnr_unpriv_06                       float64
fnr_unpriv_08                       float64
fpr_priv_06                         float64
fpr_priv_08                         float64
fpr_total_06                        float64
fpr_total_08                        float64
fpr_unpriv_06                       float64
fpr_unpriv_08                       float64
loss                                float64
precision                           float64
priv_n_06                         

In [67]:
# seeds aggregate

result_val = None
result_test = None
pattern_dev = 'eval_results_dev'
pattern_test = 'eval_results_test'
for subdir in os.listdir(f'{root}log_reg_results'):
    for file in os.listdir(f'{root}log_reg_results/{subdir}/'):
        if re.match(pattern_dev, file):
            f = pd.read_csv(
                    filepath_or_buffer=f'{root}log_reg_results/{subdir}/{file}',
                    delimiter='=',
                    header=None,
                )
            f[1] = f[1].astype(float)
            args = pd.read_json(f'{root}log_reg_results/{subdir}/args.json', typ='series', dtype=str)
            args = args.to_frame().reset_index().rename(columns={"index": 0, 0: 1})
            f = pd.concat([f, args], ignore_index=True)
            f.set_index(0, inplace=True)
            f = f.transpose()
            if result_val is None:
                result_val = f
            else:
                result_val = pd.concat([result_val, f], ignore_index=True, sort=False)
        elif re.match(pattern_test, file):
            f = pd.read_csv(
                    filepath_or_buffer=f'{root}log_reg_results/{subdir}/{file}',
                    delimiter='=',
                    header=None,
            )
            f[1] = f[1].astype(float)
            args = pd.read_json(f'{root}log_reg_results/{subdir}/args.json', typ='series', dtype=str)
            args = args.to_frame().reset_index().rename(columns={"index": 0, 0: 1})
            f = pd.concat([f, args], ignore_index=True)
            f.set_index(0, inplace=True)
            f = f.transpose()
            if result_test is None:
                result_test = f
            else:
                result_test = pd.concat([result_test, f], ignore_index=True, sort=False)
        else: pass

result_test[['acc ', 'auc_roc ', 'disparate_impact_favorable_06 ',
       'disparate_impact_favorable_08 ', 'disparate_impact_unfavorable_06 ',
       'disparate_impact_unfavorable_08 ', 'f1 ', 'fnr_priv_06 ',
       'fnr_priv_08 ', 'fnr_total_06 ', 'fnr_total_08 ', 'fnr_unpriv_06 ',
       'fnr_unpriv_08 ', 'fpr_priv_06 ', 'fpr_priv_08 ', 'fpr_total_06 ',
       'fpr_total_08 ', 'fpr_unpriv_06 ', 'fpr_unpriv_08 ', 'loss ',
       'precision ', 'priv_n_06 ', 'priv_n_08 ', 'priv_ratio_favorable_06 ',
       'priv_ratio_favorable_08 ', 'priv_ratio_unfavorable_06 ',
       'priv_ratio_unfavorable_08 ', 'priv_total_06 ', 'priv_total_08 ',
       'recall ', 'unpriv_n_06 ', 'unpriv_n_08 ', 'unpriv_ratio_favorable_06 ',
       'unpriv_ratio_favorable_08 ', 'unpriv_ratio_unfavorable_06 ',
       'unpriv_ratio_unfavorable_08 ', 'unpriv_total_06 ', 'unpriv_total_08 ']] = result_test[['acc ', 'auc_roc ', 'disparate_impact_favorable_06 ',
       'disparate_impact_favorable_08 ', 'disparate_impact_unfavorable_06 ',
       'disparate_impact_unfavorable_08 ', 'f1 ', 'fnr_priv_06 ',
       'fnr_priv_08 ', 'fnr_total_06 ', 'fnr_total_08 ', 'fnr_unpriv_06 ',
       'fnr_unpriv_08 ', 'fpr_priv_06 ', 'fpr_priv_08 ', 'fpr_total_06 ',
       'fpr_total_08 ', 'fpr_unpriv_06 ', 'fpr_unpriv_08 ', 'loss ',
       'precision ', 'priv_n_06 ', 'priv_n_08 ', 'priv_ratio_favorable_06 ',
       'priv_ratio_favorable_08 ', 'priv_ratio_unfavorable_06 ',
       'priv_ratio_unfavorable_08 ', 'priv_total_06 ', 'priv_total_08 ',
       'recall ', 'unpriv_n_06 ', 'unpriv_n_08 ', 'unpriv_ratio_favorable_06 ',
       'unpriv_ratio_favorable_08 ', 'unpriv_ratio_unfavorable_06 ',
       'unpriv_ratio_unfavorable_08 ', 'unpriv_total_06 ', 'unpriv_total_08 ',]].astype(float)

result_val[['acc ', 'auc_roc ', 'disparate_impact_favorable_06 ',
       'disparate_impact_favorable_08 ', 'disparate_impact_unfavorable_06 ',
       'disparate_impact_unfavorable_08 ', 'f1 ', 'fnr_priv_06 ',
       'fnr_priv_08 ', 'fnr_total_06 ', 'fnr_total_08 ', 'fnr_unpriv_06 ',
       'fnr_unpriv_08 ', 'fpr_priv_06 ', 'fpr_priv_08 ', 'fpr_total_06 ',
       'fpr_total_08 ', 'fpr_unpriv_06 ', 'fpr_unpriv_08 ', 'loss ',
       'precision ', 'priv_n_06 ', 'priv_n_08 ', 'priv_ratio_favorable_06 ',
       'priv_ratio_favorable_08 ', 'priv_ratio_unfavorable_06 ',
       'priv_ratio_unfavorable_08 ', 'priv_total_06 ', 'priv_total_08 ',
       'recall ', 'unpriv_n_06 ', 'unpriv_n_08 ', 'unpriv_ratio_favorable_06 ',
       'unpriv_ratio_favorable_08 ', 'unpriv_ratio_unfavorable_06 ',
       'unpriv_ratio_unfavorable_08 ', 'unpriv_total_06 ', 'unpriv_total_08 ']] = result_val[['acc ', 'auc_roc ', 'disparate_impact_favorable_06 ',
       'disparate_impact_favorable_08 ', 'disparate_impact_unfavorable_06 ',
       'disparate_impact_unfavorable_08 ', 'f1 ', 'fnr_priv_06 ',
       'fnr_priv_08 ', 'fnr_total_06 ', 'fnr_total_08 ', 'fnr_unpriv_06 ',
       'fnr_unpriv_08 ', 'fpr_priv_06 ', 'fpr_priv_08 ', 'fpr_total_06 ',
       'fpr_total_08 ', 'fpr_unpriv_06 ', 'fpr_unpriv_08 ', 'loss ',
       'precision ', 'priv_n_06 ', 'priv_n_08 ', 'priv_ratio_favorable_06 ',
       'priv_ratio_favorable_08 ', 'priv_ratio_unfavorable_06 ',
       'priv_ratio_unfavorable_08 ', 'priv_total_06 ', 'priv_total_08 ',
       'recall ', 'unpriv_n_06 ', 'unpriv_n_08 ', 'unpriv_ratio_favorable_06 ',
       'unpriv_ratio_favorable_08 ', 'unpriv_ratio_unfavorable_06 ',
       'unpriv_ratio_unfavorable_08 ', 'unpriv_total_06 ', 'unpriv_total_08 ',]].astype(float)
result_val.to_csv(f'{root}/log_reg_stats/val_results_raw.csv', index=False)
result_test.to_csv(f'{root}/log_reg_stats/test_results_raw.csv', index=False)


In [27]:
result_val

,acc,auc_roc,disparate_impact_favorable_06,disparate_impact_favorable_08,disparate_impact_unfavorable_06,disparate_impact_unfavorable_08,f1,fnr_priv_06,fnr_priv_08,fnr_total_06,...,batch_size,epochs,max_seq_length,learning_rate,reg_strength,class_weight,output_dir,do_train,do_eval,test
0,0.879241,0.88808,0.161209,0.121588,1.18623,1.16135,0.927725,0.0692943,0.0594458,0.0583906,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
1,0.876817,0.8932,0.210575,0.109897,1.19575,1.1845,0.925555,0.0813732,0.0710327,0.0696762,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
2,0.878433,0.889583,0.19658,0.217183,1.20363,1.16462,0.926424,0.0826446,0.0710327,0.0701668,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
3,0.880048,0.892417,0.194653,0.215139,1.20666,1.16695,0.927331,0.0820089,0.0710327,0.0701668,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
4,0.88126,0.891925,0.190506,0,1.18581,1.19183,0.92871,0.0705658,0.0619647,0.0603533,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.937041,0.755284,1.00671,1.01341,0.508529,0,0.245648,0.848558,0.853009,0.854191,...,64,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
146,0.937041,0.755591,1.00706,1.01374,0.49605,0,0.248555,0.846154,0.850694,0.851894,...,64,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
147,0.93696,0.755047,1.00662,1.01332,0.511748,0,0.24395,0.84976,0.854167,0.855339,...,64,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
148,0.937041,0.755537,1.00758,1.01424,0.478439,0,0.252874,0.842548,0.847222,0.84845,...,64,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True


In [28]:
result_test

,acc,auc_roc,disparate_impact_favorable_06,disparate_impact_favorable_08,disparate_impact_unfavorable_06,disparate_impact_unfavorable_08,f1,fnr_priv_06,fnr_priv_08,fnr_total_06,...,batch_size,epochs,max_seq_length,learning_rate,reg_strength,class_weight,output_dir,do_train,do_eval,test
0,0.886235,0.90999,0.160013,0,1.1773,1.17509,0.932468,0.0647851,0.0547112,0.0527344,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
1,0.887045,0.908225,0.147937,0,1.19794,1.19145,0.932495,0.0731238,0.0612969,0.059082,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
2,0.887854,0.90499,0.173556,0,1.20035,1.201,0.932718,0.0769724,0.064843,0.0625,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
3,0.889474,0.906283,0.174464,0,1.19884,1.1998,0.933722,0.0744067,0.0633232,0.0610352,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
4,0.888664,0.906275,0.142358,0,1.19071,1.18321,0.933687,0.0679923,0.0567376,0.0546875,...,32,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.936557,0.722813,0.985477,0.982049,2.19436,2.40919,0.235409,0.864897,0.861432,0.861079,...,64,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
146,0.936234,0.723528,0.985815,0.982372,2.13424,2.34858,0.234496,0.864897,0.861432,0.861079,...,64,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
147,0.936557,0.723133,0.985477,0.982049,2.19436,2.40919,0.235409,0.864897,0.861432,0.861079,...,64,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True
148,0.936395,0.723617,0.985984,0.982534,2.1054,2.31941,0.237911,0.862485,0.859122,0.858783,...,64,1000,128,0.002,0,none,/nethome/mhalevy3/HateSpeech/benchmarking/base...,False,False,True


In [29]:
result_val.columns

Index(['acc ', 'auc_roc ', 'disparate_impact_favorable_06 ',
       'disparate_impact_favorable_08 ', 'disparate_impact_unfavorable_06 ',
       'disparate_impact_unfavorable_08 ', 'f1 ', 'fnr_priv_06 ',
       'fnr_priv_08 ', 'fnr_total_06 ', 'fnr_total_08 ', 'fnr_unpriv_06 ',
       'fnr_unpriv_08 ', 'fpr_priv_06 ', 'fpr_priv_08 ', 'fpr_total_06 ',
       'fpr_total_08 ', 'fpr_unpriv_06 ', 'fpr_unpriv_08 ', 'loss ',
       'precision ', 'priv_n_06 ', 'priv_n_08 ', 'priv_ratio_favorable_06 ',
       'priv_ratio_favorable_08 ', 'priv_ratio_unfavorable_06 ',
       'priv_ratio_unfavorable_08 ', 'priv_total_06 ', 'priv_total_08 ',
       'recall ', 'unpriv_n_06 ', 'unpriv_n_08 ', 'unpriv_ratio_favorable_06 ',
       'unpriv_ratio_favorable_08 ', 'unpriv_ratio_unfavorable_06 ',
       'unpriv_ratio_unfavorable_08 ', 'unpriv_total_06 ', 'unpriv_total_08 ',
       'is_local', 'is_gcloud', 'is_gridsearch', 'retrain', 'task_name',
       'seed', 'dataset', 'batch_size', 'epochs', 'max_seq_len

In [32]:
def ci_sample(series):
    pop_mean = series.mean()
    pop_std = series.std(ddof=0)
    pop_n =  len(series) - 1
    return pop_mean - (1.96*pop_std/math.sqrt(pop_n)), pop_mean + (1.96*pop_std/math.sqrt(pop_n))

In [54]:
result_val.dtypes


0
acc                                 object
auc_roc                             object
disparate_impact_favorable_06       object
disparate_impact_favorable_08       object
disparate_impact_unfavorable_06     object
disparate_impact_unfavorable_08     object
f1                                  object
fnr_priv_06                         object
fnr_priv_08                         object
fnr_total_06                        object
fnr_total_08                        object
fnr_unpriv_06                       object
fnr_unpriv_08                       object
fpr_priv_06                         object
fpr_priv_08                         object
fpr_total_06                        object
fpr_total_08                        object
fpr_unpriv_06                       object
fpr_unpriv_08                       object
loss                                object
precision                           object
priv_n_06                           object
priv_n_08                           object
priv_rati

In [70]:
temp_dev = result_val.groupby(['dataset', 'task_name']).agg(
    acc_mean=('acc ', 'mean'),
    acc_std=('acc ', 'std'),
    acc_ci=('acc ', ci_sample),
    auc_roc_mean=('auc_roc ', 'mean'),
    auc_roc_std=('auc_roc ', 'std'),
    auc_roc_ci=('auc_roc ', ci_sample),
    f1_mean=('f1 ', 'mean'),
    f1_std=('f1 ', 'std'),
    f1_ci=('f1 ', ci_sample),
    precision_mean=('precision ', 'mean'),
    precision_std=('precision ', 'std'),
    precision_ci=('precision ', ci_sample),
    recall_mean=('recall ', 'mean'),
    recall_std=('recall ', 'std'),
    recall_ci=('recall ', ci_sample),
    disparate_impact_favorable_06_mean=('disparate_impact_favorable_06 ', np.mean),
    disparate_impact_favorable_06_std=('disparate_impact_favorable_06 ', np.std),
    disparate_impact_favorable_06_ci=('disparate_impact_favorable_06 ',ci_sample),
    disparate_impact_favorable_08_mean=('disparate_impact_favorable_08 ', np.mean),
    disparate_impact_favorable_08_std=('disparate_impact_favorable_08 ',np.std),
    disparate_impact_favorable_08_ci=('disparate_impact_favorable_08 ', ci_sample),
    disparate_impact_unfavorable_06_mean=('disparate_impact_unfavorable_06 ', np.mean),
    disparate_impact_unfavorable_06_std=('disparate_impact_unfavorable_06 ', np.std),
    disparate_impact_unfavorable_06_ci=('disparate_impact_unfavorable_06 ',ci_sample),
    disparate_impact_unfavorable_08_mean=('disparate_impact_unfavorable_08 ', np.mean),
    disparate_impact_unfavorable_08_std=('disparate_impact_unfavorable_08 ',np.std),
    disparate_impact_unfavorable_08_ci=('disparate_impact_unfavorable_08 ', ci_sample),
    fnr_priv_06_mean=('fnr_priv_06 ', np.mean),
    fnr_priv_06_std=('fnr_priv_06 ', np.std),
    fnr_priv_06_ci=('fnr_priv_06 ', ci_sample),
    fnr_priv_08_mean=('fnr_priv_08 ', np.mean),
    fnr_priv_08_std=('fnr_priv_08 ', np.std),
    fnr_priv_08_ci=('fnr_priv_08 ', ci_sample),
    fnr_total_06_mean=('fnr_total_06 ', np.mean),
    fnr_total_06_std=('fnr_total_06 ', np.std),
    fnr_total_06_ci=('fnr_total_06 ', ci_sample),
    fnr_total_08_mean=('fnr_total_08 ', np.mean),
    fnr_total_08_std=('fnr_total_08 ', np.std),
    fnr_total_08_ci=('fnr_total_08 ', ci_sample),
    fnr_unpriv_06_mean=('fnr_unpriv_06 ', np.mean),
    fnr_unpriv_06_std=('fnr_unpriv_06 ', np.std),
    fnr_unpriv_06_ci=('fnr_unpriv_06 ', ci_sample),
    fnr_unpriv_08_mean=('fnr_unpriv_08 ', np.mean),
    fnr_unpriv_08_std=('fnr_unpriv_08 ', np.std),
    fnr_unpriv_08_ci=('fnr_unpriv_08 ', ci_sample),
    fpr_priv_06_mean=('fpr_priv_06 ', np.mean),
    fpr_priv_06_std=('fpr_priv_06 ', np.std),
    fpr_priv_06_ci=('fpr_priv_06 ', ci_sample),
    fpr_priv_08_mean=('fpr_priv_08 ', np.mean),
    fpr_priv_08_std=('fpr_priv_08 ', np.std),
    fpr_priv_08_ci=('fpr_priv_08 ', ci_sample),
    fpr_total_06_mean=('fpr_total_06 ', np.mean),
    fpr_total_06_std=('fpr_total_06 ', np.std),
    fpr_total_06_ci=('fpr_total_06 ', ci_sample),
    fpr_total_08_mean=('fpr_total_08 ', np.mean),
    fpr_total_08_std=('fpr_total_08 ', np.std),
    fpr_total_08_ci=('fpr_total_08 ', ci_sample),
    fpr_unpriv_06_mean=('fpr_unpriv_06 ', np.mean),
    fpr_unpriv_06_std=('fpr_unpriv_06 ', np.std),
    fpr_unpriv_06_ci=('fpr_unpriv_06 ', ci_sample),
    fpr_unpriv_08_mean=('fpr_unpriv_08 ', np.mean),
    fpr_unpriv_08_std=('fpr_unpriv_08 ', np.std),
    fpr_unpriv_08_ci=('fpr_unpriv_08 ', ci_sample),
    priv_n_06=('priv_n_06 ', np.median),
    priv_n_08=('priv_n_08 ', np.median),
    unpriv_n_06=('unpriv_n_06 ', np.median),
    unpriv_n_08=('unpriv_n_08 ', np.median),
).reset_index().sort_values(by=['dataset', 'task_name'])
temp_dev.to_csv(f'{root}/log_reg_stats/dev_agg_seed_results.csv', index=False)
temp_dev

,dataset,task_name,acc_mean,acc_std,acc_ci,auc_roc_mean,auc_roc_std,auc_roc_ci,f1_mean,f1_std,...,fpr_unpriv_06_mean,fpr_unpriv_06_std,fpr_unpriv_06_ci,fpr_unpriv_08_mean,fpr_unpriv_08_std,fpr_unpriv_08_ci,priv_n_06,priv_n_08,unpriv_n_06,unpriv_n_08
0,davidson,glove,0.879564,0.001636,"(0.8785500579036941, 0.8805775470577012)",0.890331,0.002222,"(0.8889534336637371, 0.891708173124898)",0.927512,0.001172,...,0.607692,0.076495,"(0.5602805050799206, 0.6551041103046947)",0.0,0.0,"(0.0, 0.0)",478.0,53.0,1998.0,2423.0
1,davidson,ngram,0.942569,0.001422,"(0.9416871656077274, 0.9434501420361631)",0.975600,0.000472,"(0.9753069607280388, 0.9758921178318366)",0.965446,0.000813,...,0.384615,0.000000,"(0.38461538461538464, 0.38461538461538464)",0.0,0.0,"(0.0, 0.0)",478.0,53.0,1998.0,2423.0
2,davidson,tf_idf,0.875121,0.000847,"(0.8745960717061076, 0.8756462569376913)",0.910278,0.000839,"(0.9097577729885638, 0.910797615938896)",0.927679,0.000477,...,0.846154,0.000000,"(0.8461538461538461, 0.8461538461538461)",0.0,0.0,"(0.0, 0.0)",478.0,53.0,1998.0,2423.0
3,founta,glove,0.897611,0.000834,"(0.8970938947205397, 0.8981280777449755)",0.932995,0.000249,"(0.9328408853341235, 0.9331496341894971)",0.806685,0.002951,...,0.500000,0.000000,"(0.5, 0.5)",0.0,0.0,"(0.0, 0.0)",139.0,10.0,9028.0,9157.0
4,founta,ngram,0.928777,0.000284,"(0.9286012626306578, 0.9289530181272462)",0.955304,0.000134,"(0.9552211418682848, 0.9553866490786758)",0.866708,0.000676,...,0.337500,0.013176,"(0.32933333333333337, 0.3456666666666667)",0.0,0.0,"(0.0, 0.0)",139.0,10.0,9028.0,9157.0
5,founta,tf_idf,0.893553,0.000328,"(0.8933493719192672, 0.8937565579322647)",0.914120,0.000489,"(0.9138168199037415, 0.914422430279364)",0.786157,0.000982,...,0.300000,0.032867,"(0.27962875422756417, 0.3203712457724359)",0.0,0.0,"(0.0, 0.0)",139.0,10.0,9028.0,9157.0
6,golbeck,glove,0.759959,0.001538,"(0.759005394838842, 0.760912260152308)",0.589601,0.005645,"(0.5861018409905066, 0.5930997977080714)",0.052724,0.011394,...,0.000000,0.000000,"(0.0, 0.0)",0.0,0.0,"(0.0, 0.0)",8.0,1.0,1935.0,1942.0
7,golbeck,ngram,0.774730,0.001031,"(0.7740909087029919, 0.7753686679037586)",0.653591,0.000959,"(0.6529971964843347, 0.6541857354156896)",0.214033,0.004304,...,0.000000,0.000000,"(0.0, 0.0)",0.0,0.0,"(0.0, 0.0)",8.0,1.0,1935.0,1942.0
8,golbeck,tf_idf,0.763973,0.000497,"(0.7636650472457931, 0.7642814249221757)",0.604567,0.000637,"(0.6041717845499798, 0.6049612682759479)",0.180735,0.006150,...,0.000000,0.000000,"(0.0, 0.0)",0.0,0.0,"(0.0, 0.0)",8.0,1.0,1935.0,1942.0
9,harassment,glove,0.834568,0.000679,"(0.8341469976975502, 0.8349888231680809)",0.899041,0.000188,"(0.8989244358581123, 0.8991577370125237)",0.782852,0.001256,...,0.572093,0.012009,"(0.5646496125645659, 0.5795364339470621)",0.6,0.0,"(0.5999999999999998, 0.6)",648.0,79.0,12798.0,13367.0


In [69]:
temp_test = result_test.groupby(['dataset', 'task_name']).agg(
    acc_mean=('acc ', 'mean'),
    acc_std=('acc ', 'std'),
    acc_ci=('acc ', ci_sample),
    auc_roc_mean=('auc_roc ', 'mean'),
    auc_roc_std=('auc_roc ', 'std'),
    auc_roc_ci=('auc_roc ', ci_sample),
    f1_mean=('f1 ', 'mean'),
    f1_std=('f1 ', 'std'),
    f1_ci=('f1 ', ci_sample),
    precision_mean=('precision ', 'mean'),
    precision_std=('precision ', 'std'),
    precision_ci=('precision ', ci_sample),
    recall_mean=('recall ', 'mean'),
    recall_std=('recall ', 'std'),
    recall_ci=('recall ', ci_sample),
    disparate_impact_favorable_06_mean=('disparate_impact_favorable_06 ', np.mean),
    disparate_impact_favorable_06_std=('disparate_impact_favorable_06 ', np.std),
    disparate_impact_favorable_06_ci=('disparate_impact_favorable_06 ',ci_sample),
    disparate_impact_favorable_08_mean=('disparate_impact_favorable_08 ', np.mean),
    disparate_impact_favorable_08_std=('disparate_impact_favorable_08 ',np.std),
    disparate_impact_favorable_08_ci=('disparate_impact_favorable_08 ', ci_sample),
    disparate_impact_unfavorable_06_mean=('disparate_impact_unfavorable_06 ', np.mean),
    disparate_impact_unfavorable_06_std=('disparate_impact_unfavorable_06 ', np.std),
    disparate_impact_unfavorable_06_ci=('disparate_impact_unfavorable_06 ',ci_sample),
    disparate_impact_unfavorable_08_mean=('disparate_impact_unfavorable_08 ', np.mean),
    disparate_impact_unfavorable_08_std=('disparate_impact_unfavorable_08 ',np.std),
    disparate_impact_unfavorable_08_ci=('disparate_impact_unfavorable_08 ', ci_sample),
    fnr_priv_06_mean=('fnr_priv_06 ', np.mean),
    fnr_priv_06_std=('fnr_priv_06 ', np.std),
    fnr_priv_06_ci=('fnr_priv_06 ', ci_sample),
    fnr_priv_08_mean=('fnr_priv_08 ', np.mean),
    fnr_priv_08_std=('fnr_priv_08 ', np.std),
    fnr_priv_08_ci=('fnr_priv_08 ', ci_sample),
    fnr_total_06_mean=('fnr_total_06 ', np.mean),
    fnr_total_06_std=('fnr_total_06 ', np.std),
    fnr_total_06_ci=('fnr_total_06 ', ci_sample),
    fnr_total_08_mean=('fnr_total_08 ', np.mean),
    fnr_total_08_std=('fnr_total_08 ', np.std),
    fnr_total_08_ci=('fnr_total_08 ', ci_sample),
    fnr_unpriv_06_mean=('fnr_unpriv_06 ', np.mean),
    fnr_unpriv_06_std=('fnr_unpriv_06 ', np.std),
    fnr_unpriv_06_ci=('fnr_unpriv_06 ', ci_sample),
    fnr_unpriv_08_mean=('fnr_unpriv_08 ', np.mean),
    fnr_unpriv_08_std=('fnr_unpriv_08 ', np.std),
    fnr_unpriv_08_ci=('fnr_unpriv_08 ', ci_sample),
    fpr_priv_06_mean=('fpr_priv_06 ', np.mean),
    fpr_priv_06_std=('fpr_priv_06 ', np.std),
    fpr_priv_06_ci=('fpr_priv_06 ', ci_sample),
    fpr_priv_08_mean=('fpr_priv_08 ', np.mean),
    fpr_priv_08_std=('fpr_priv_08 ', np.std),
    fpr_priv_08_ci=('fpr_priv_08 ', ci_sample),
    fpr_total_06_mean=('fpr_total_06 ', np.mean),
    fpr_total_06_std=('fpr_total_06 ', np.std),
    fpr_total_06_ci=('fpr_total_06 ', ci_sample),
    fpr_total_08_mean=('fpr_total_08 ', np.mean),
    fpr_total_08_std=('fpr_total_08 ', np.std),
    fpr_total_08_ci=('fpr_total_08 ', ci_sample),
    fpr_unpriv_06_mean=('fpr_unpriv_06 ', np.mean),
    fpr_unpriv_06_std=('fpr_unpriv_06 ', np.std),
    fpr_unpriv_06_ci=('fpr_unpriv_06 ', ci_sample),
    fpr_unpriv_08_mean=('fpr_unpriv_08 ', np.mean),
    fpr_unpriv_08_std=('fpr_unpriv_08 ', np.std),
    fpr_unpriv_08_ci=('fpr_unpriv_08 ', ci_sample),
    priv_n_06=('priv_n_06 ', np.median),
    priv_n_08=('priv_n_08 ', np.median),
    unpriv_n_06=('unpriv_n_06 ', np.median),
    unpriv_n_08=('unpriv_n_08 ', np.median),
).reset_index().sort_values(by=['dataset', 'task_name'])
temp_test.to_csv(f'{root}/log_reg_stats/test_agg_seed_results.csv', index=False)
temp_test

,dataset,task_name,acc_mean,acc_std,acc_ci,auc_roc_mean,auc_roc_std,auc_roc_ci,f1_mean,f1_std,...,fpr_unpriv_06_mean,fpr_unpriv_06_std,fpr_unpriv_06_ci,fpr_unpriv_08_mean,fpr_unpriv_08_std,fpr_unpriv_08_ci,priv_n_06,priv_n_08,unpriv_n_06,unpriv_n_08
0,davidson,glove,0.887328,0.001309,"(0.8865165472814914, 0.8881393312623623)",0.907370,0.001982,"(0.9061414733989916, 0.9085987910167492)",0.932861,0.000617,...,0.446154,0.079446,"(0.39691282158097413, 0.49539487072671834)",0.000000,0.000000,"(0.0, 0.0)",502.0,74.0,1968.0,2396.0
1,davidson,ngram,0.939312,0.000982,"(0.9387033864172455, 0.939920111068869)",0.977848,0.000251,"(0.9776921062772947, 0.9780029387170595)",0.963891,0.000545,...,0.392308,0.024325,"(0.37723076923076926, 0.40738461538461546)",0.000000,0.000000,"(0.0, 0.0)",502.0,74.0,1968.0,2396.0
2,davidson,tf_idf,0.871943,0.000333,"(0.871736729002732, 0.8721499086849511)",0.885099,0.000678,"(0.8846788297635392, 0.885519479658)",0.926375,0.000190,...,0.846154,0.000000,"(0.8461538461538461, 0.8461538461538461)",0.000000,0.000000,"(0.0, 0.0)",502.0,74.0,1968.0,2396.0
3,founta,glove,0.899826,0.000561,"(0.8994781086922924, 0.9001732518194875)",0.935428,0.000305,"(0.9352383406918522, 0.935616776152325)",0.808424,0.002534,...,0.294118,0.000000,"(0.29411764705882343, 0.29411764705882354)",0.000000,0.000000,"(0.0, 0.0)",117.0,7.0,9061.0,9171.0
4,founta,ngram,0.935095,0.000388,"(0.9348543425436208, 0.9353352526788524)",0.959789,0.000147,"(0.9596976988256661, 0.9598804242669853)",0.877733,0.000623,...,0.235294,0.000000,"(0.2352941176470588, 0.2352941176470588)",0.000000,0.000000,"(0.0, 0.0)",117.0,7.0,9061.0,9171.0
5,founta,tf_idf,0.895674,0.000542,"(0.8953386787548121, 0.896010207735056)",0.914075,0.000249,"(0.9139211009213997, 0.9142291595270562)",0.787913,0.001371,...,0.294118,0.000000,"(0.29411764705882343, 0.29411764705882354)",0.000000,0.000000,"(0.0, 0.0)",117.0,7.0,9061.0,9171.0
6,golbeck,glove,0.759383,0.000939,"(0.7588012211568581, 0.7599648244135154)",0.587727,0.004376,"(0.5850141425823744, 0.590439031531853)",0.048368,0.006537,...,0.000000,0.000000,"(0.0, 0.0)",NaN,NaN,"(nan, nan)",13.0,NaN,1932.0,NaN
7,golbeck,ngram,0.779177,0.000737,"(0.7787204922927863, 0.7796342550025933)",0.683468,0.000562,"(0.6831197540509362, 0.6838166554224829)",0.272135,0.005568,...,0.000000,0.000000,"(0.0, 0.0)",NaN,NaN,"(nan, nan)",13.0,NaN,1932.0,NaN
8,golbeck,tf_idf,0.776555,0.000434,"(0.7762865351041158, 0.7768239928881328)",0.639059,0.000607,"(0.6386833417438993, 0.6394356262660494)",0.261335,0.006933,...,0.000000,0.000000,"(0.0, 0.0)",NaN,NaN,"(nan, nan)",13.0,NaN,1932.0,NaN
9,harassment,glove,0.839305,0.000563,"(0.8389562937668006, 0.8396544364044031)",0.902648,0.000171,"(0.9025419199844602, 0.9027544176200626)",0.786571,0.001216,...,0.607692,0.016217,"(0.5976410256410256, 0.6177435897435897)",1.000000,0.000000,"(1.0, 1.0)",596.0,70.0,12850.0,13376.0


In [38]:
temp.to_csv('../data/contextualize_results/stats_on_tasks.csv', index=False)

In [ ]:
models_dict = {'twitter': {'twitter_es_reg_nb5_h5_is_bal_pos_seed_', 'twitter_es_reg_nb0_h1_bal_seed_', 'twitter_es_vanilla_bal_seed_'},
                'twitter_harass': {'twitter_harass_es_reg_nb5_h5_is_bal_pos_seed_','twitter_harass_es_reg_nb0_h1_bal_seed_','twitter_harass_es_vanilla_bal_seed_'},
               'gab': {'majority_gab_es_vanilla_bal_seed_', 'majority_gab_es_reg_nb0_h1_bal_seed_', 'majority_gab_es_reg_nb5_h5_is_bal_pos_seed_'},
               'ws': {'ws_es_vanilla_bal_seed_', 'ws_es_reg_nb0_h1_bal_seed_','ws_es_reg_nb5_h5_is_bal_pos_seed_'},
               'nyt': {},
               }

In [33]:
task_to_learner_results = None
for task in models_dict.keys():
    if task_to_learner_results is None:
        task_to_learner_results = temp[(temp['model'].isin(models_dict[task])) & (temp['task'] == task)]
    else:
        task_to_learner_results = pd.concat([task_to_learner_results, temp[(temp['model'].isin(models_dict[task])) & (temp['task'] == task)]])
task_to_learner_results

,model,task,acc_mean,acc_std,acc_ci,auc_roc_mean,auc_roc_std,auc_roc_ci,f1_mean,f1_std,...,disparate_impact_06_mean,disparate_impact_06_std,disparate_impact_06_ci,disparate_impact_08_mean,disparate_impact_08_std,disparate_impact_08_ci,priv_n_06,priv_n_08,unpriv_n_06,unpriv_n_08
14,twitter_es_reg_nb0_h1_bal_seed_,twitter,0.740416,0.013804,"(0.7318604701540944, 0.7489718244091427)",0.851023,0.005828,"(0.8474105131000839, 0.8546345486463854)",0.400923,0.009749,...,0.857148,0.044112,"(0.8298066061381789, 0.884489005007136)",0.962671,0.097553,"(0.902206724586391, 1.023134586881783)",525.0,64.0,9904.0,10365.0
18,twitter_es_reg_nb5_h5_is_bal_pos_seed_,twitter,0.780775,0.020300,"(0.7681924281851983, 0.7933570971767733)",0.861636,0.006482,"(0.8576180444353265, 0.8656529654302911)",0.434504,0.012329,...,0.826937,0.060878,"(0.7892042012454198, 0.8646696652235802)",0.866940,0.114318,"(0.7960850410549574, 0.9377945217594896)",525.0,64.0,9904.0,10365.0
22,twitter_es_vanilla_bal_seed_,twitter,0.797047,0.016549,"(0.786789594562518, 0.8073037988596702)",0.867681,0.012040,"(0.8602187635615669, 0.8751441492098829)",0.452028,0.014638,...,0.840683,0.035621,"(0.8186045992182596, 0.8627610571687874)",0.874632,0.077094,"(0.8268481619460178, 0.9224149791085426)",525.0,64.0,9904.0,10365.0
27,twitter_harass_es_reg_nb0_h1_bal_seed_,twitter_harass,0.699126,0.108810,"(0.6316849387126151, 0.7665673015302938)",0.913960,0.096742,"(0.8539989489968628, 0.9739213422284224)",0.724786,0.059479,...,inf,NaN,"(nan, nan)",inf,NaN,"(nan, nan)",656.0,77.0,12847.0,13426.0
32,twitter_harass_es_reg_nb5_h5_is_bal_pos_seed_,twitter_harass,0.798674,0.010532,"(0.7921467442694817, 0.8052019930481517)",0.951445,0.004280,"(0.9487928326630026, 0.9540978517512333)",0.790119,0.007984,...,41.367705,8.509532,"(36.09344237057658, 46.64196711913195)",inf,NaN,"(nan, nan)",656.0,77.0,12847.0,13426.0
37,twitter_harass_es_vanilla_bal_seed_,twitter_harass,0.802177,0.013758,"(0.7936501224867812, 0.8107044653825811)",0.951767,0.005010,"(0.9486616697289599, 0.9548716952451467)",0.793076,0.010919,...,35.196131,11.226862,"(28.23764979208232, 42.15461160470309)",inf,NaN,"(nan, nan)",656.0,77.0,12847.0,13426.0
0,majority_gab_es_reg_nb0_h1_bal_seed_,gab,0.862108,0.012026,"(0.8546544856593332, 0.8695623818105465)",0.874507,0.007422,"(0.8699068385251156, 0.8791072046781914)",0.449921,0.018215,...,1.581563,0.638837,"(1.1856077090048187, 1.9775184225582472)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
4,majority_gab_es_reg_nb5_h5_is_bal_pos_seed_,gab,0.881928,0.020701,"(0.8690969473006679, 0.8947584743860789)",0.886380,0.010175,"(0.8800729619505216, 0.8926863453965002)",0.480698,0.030686,...,1.286723,0.038792,"(1.2626797696530172, 1.3107662170699763)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
8,majority_gab_es_vanilla_bal_seed_,gab,0.882831,0.015758,"(0.8730642575235151, 0.8925983930788944)",0.891738,0.007496,"(0.8870917266629993, 0.8963837414831942)",0.488232,0.021547,...,1.582046,0.701131,"(1.1474801485394046, 2.0166115835064615)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
42,ws_es_reg_nb0_h1_bal_seed_,ws,0.895282,0.012831,"(0.8873287167761869, 0.9032344491294447)",0.880330,0.009994,"(0.8741355728690714, 0.8865244461242809)",0.547665,0.020910,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN


In [34]:
other_tasks_results = temp[(temp['model'].isin(models_dict['twitter'])) & (temp['task'].isin({'gab', 'ws', 'twitter_harass', 'nyt'}))]
other_tasks_results = pd.concat([other_tasks_results, temp[(temp['model'].isin(models_dict['gab'])) & (temp['task'].isin({'twitter', 'twitter_harass','ws', 'nyt'}))]])
other_tasks_results = pd.concat([other_tasks_results, temp[(temp['model'].isin(models_dict['twitter_harass'])) & (temp['task'].isin({'twitter', 'gab', 'ws', 'nyt'}))]])
other_tasks_results = pd.concat([other_tasks_results, temp[(temp['model'].isin(models_dict['ws'])) & (temp['task'].isin({'twitter', 'twitter_harass','gab', 'nyt'}))]])

In [35]:
other_tasks_results

,model,task,acc_mean,acc_std,acc_ci,auc_roc_mean,auc_roc_std,auc_roc_ci,f1_mean,f1_std,...,disparate_impact_06_mean,disparate_impact_06_std,disparate_impact_06_ci,disparate_impact_08_mean,disparate_impact_08_std,disparate_impact_08_ci,priv_n_06,priv_n_08,unpriv_n_06,unpriv_n_08
12,twitter_es_reg_nb0_h1_bal_seed_,gab,0.628976,0.036343,"(0.6064502124205815, 0.651501594808334)",0.818889,0.007008,"(0.8145454343478087, 0.8232331002209861)",0.276995,0.014425,...,0.856639,0.060978,"(0.8188442563563496, 0.8944327502821539)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
13,twitter_es_reg_nb0_h1_bal_seed_,nyt,0.692862,0.067703,"(0.6508996380141923, 0.7348249996669673)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
15,twitter_es_reg_nb0_h1_bal_seed_,ws,0.742009,0.025167,"(0.7264106936761919, 0.7576075711639908)",0.748519,0.014860,"(0.739308099673421, 0.7577289373636157)",0.333568,0.027191,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
16,twitter_es_reg_nb5_h5_is_bal_pos_seed_,gab,0.697771,0.047310,"(0.6684477917724448, 0.7270943769022539)",0.821897,0.006244,"(0.8180272449425315, 0.8257673052161579)",0.310001,0.022413,...,1.219403,0.461532,"(0.9333420557721496, 1.5054630136303855)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
17,twitter_es_reg_nb5_h5_is_bal_pos_seed_,nyt,0.794993,0.068235,"(0.7527000530041628, 0.8372854542422141)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
19,twitter_es_reg_nb5_h5_is_bal_pos_seed_,ws,0.771081,0.030103,"(0.7524226540514041, 0.7897386853702091)",0.750957,0.017452,"(0.740140144293877, 0.7617734359530367)",0.336322,0.026036,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
20,twitter_es_vanilla_bal_seed_,gab,0.766506,0.026450,"(0.7501121545689884, 0.7828998936237828)",0.856527,0.003998,"(0.8540483260302403, 0.8590049000279665)",0.365528,0.015665,...,1.142336,0.538762,"(0.8084076131365296, 1.4762634791990161)",inf,NaN,"(nan, nan)",3.0,1.0,1657.0,1659.0
21,twitter_es_vanilla_bal_seed_,nyt,0.639087,0.088046,"(0.5845155413088085, 0.6936583717346699)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
23,twitter_es_vanilla_bal_seed_,ws,0.770015,0.025544,"(0.7541827073545508, 0.7858477340457535)",0.781363,0.007915,"(0.7764569630741037, 0.7862685829847756)",0.367121,0.020248,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN
1,majority_gab_es_reg_nb0_h1_bal_seed_,nyt,0.917362,0.024158,"(0.9023890204927413, 0.9323356171884182)",0.000000,0.000000,"(0.0, 0.0)",0.000000,0.000000,...,NaN,NaN,"(nan, nan)",NaN,NaN,"(nan, nan)",NaN,NaN,NaN,NaN


In [37]:
task_to_learner_results.to_csv('../data/contextualize_results/task_to_learner.csv', index=False)
other_tasks_results.to_csv('../data/contextualize_results/other_tasks_with_learner.csv', index=False)
